In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
from IPython.display import display, HTML
import seaborn as sns
import random
import matplotlib.pyplot as plt
import plotly.express as px
import pandas_profiling as pp
from scipy import stats
from scipy.stats import norm
%matplotlib inline
import h2o
from h2o.automl import H2OAutoML

In [2]:
df = pd.read_csv('data2/landmarks.csv')
df.columns = [f'col{i}' for i in range(1405)]
df.astype({'col1404':'string'})
train, validate, test = np.split(df, [int(.7*len(df)), int(.85*len(df))])

In [3]:
train

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col1395,col1396,col1397,col1398,col1399,col1400,col1401,col1402,col1403,col1404
0,0.506777,0.685168,-0.081059,0.508899,0.589536,-0.155494,0.507721,0.619720,-0.081626,0.481201,...,0.547818,0.421041,-0.017888,0.701051,0.395390,0.041529,0.714997,0.388807,0.042386,contempt
1,0.522592,0.728219,-0.069704,0.563158,0.622856,-0.154344,0.532915,0.651415,-0.075771,0.527397,...,0.563678,0.440509,-0.014150,0.693491,0.415474,0.097632,0.706497,0.406117,0.102601,contempt
2,0.507426,0.711682,-0.081520,0.490157,0.622659,-0.162877,0.500488,0.650763,-0.086197,0.465061,...,0.549762,0.444901,-0.025063,0.724568,0.406819,0.017024,0.740692,0.396217,0.016287,contempt
3,0.484002,0.716097,-0.095621,0.490276,0.590272,-0.168015,0.489010,0.631962,-0.094333,0.468822,...,0.548870,0.438363,-0.009491,0.714356,0.434477,0.052216,0.732640,0.424303,0.053430,contempt
4,0.534178,0.709634,-0.044477,0.526397,0.637113,-0.133007,0.526363,0.654088,-0.060292,0.489256,...,0.537986,0.445891,-0.039478,0.672400,0.397646,0.011078,0.682771,0.388529,0.011500,contempt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20186,0.472416,0.831119,-0.078009,0.433333,0.683983,-0.205998,0.469413,0.721407,-0.095350,0.418504,...,0.566312,0.398660,-0.075605,0.835252,0.338184,-0.069541,0.855527,0.327214,-0.076961,anger
20187,0.508826,0.647192,-0.132496,0.508906,0.545129,-0.182883,0.507683,0.586804,-0.118799,0.480202,...,0.558486,0.437194,0.011042,0.720975,0.432556,0.079688,0.741551,0.415384,0.083271,anger
20188,0.492575,0.709858,-0.075782,0.496822,0.612935,-0.156433,0.495564,0.640419,-0.076693,0.476808,...,0.555693,0.442941,-0.030559,0.722812,0.415416,0.031063,0.737344,0.407947,0.031359,anger
20189,0.490839,0.679184,-0.071439,0.517260,0.609654,-0.155441,0.501953,0.631603,-0.078049,0.496043,...,0.559266,0.452765,-0.026479,0.708093,0.437545,0.054406,0.723188,0.427620,0.057660,anger


In [4]:
h2o.init(
    nthreads=8,     # number of threads when launching a new H2O server
    max_mem_size=8  # in gigabytes
)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /home/vorkov/Workspace/Emotion-Decetion-Service/venv/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp6l0gx1ws
  JVM stdout: /tmp/tmp6l0gx1ws/h2o_vorkov_started_from_python.out
  JVM stderr: /tmp/tmp6l0gx1ws/h2o_vorkov_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Moscow
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_vorkov_s0lk7n
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [5]:
train_h2o_frame = h2o.H2OFrame(train)
test_h2o_frame = h2o.H2OFrame(test)

x = train_h2o_frame.columns
y = "col1404"
x.remove(y)

aml = H2OAutoML(max_models=8, seed=1, max_runtime_secs=60*60, verbosity='info')
aml.train(x=x, y=y, training_frame=train_h2o_frame)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
00:04:42.38: Project: AutoML_1_20230909_00442
00:04:42.39: 5-fold cross-validation will be used.
00:04:42.40: Setting stopping tolerance adaptively based on the training frame: 0.007037543391537052
00:04:42.40: Build control seed: 1
00:04:42.42: training frame: Frame key: AutoML_1_20230909_00442_training_Key_Frame__upload_9b0ad4f7d1eb0254f20700a6c8f745c0.hex    cols: 1405    rows: 20191  chunks: 129    size: 239163670  checksum: 7326891014961461210
00:04:42.42: validation frame: NULL
00:04:42.42: leaderboard frame: NULL
00:04:42.42: blending frame: NULL
00:04:42.42: response column: col1404
00:04:42.42: fold column: null
00:04:42.42: weights column: null
00:04:42.53: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_searc

Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_1_AutoML_1_20230909_00442


GLM Model: summary
    family       link         regularization               lambda_search                                                                   number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  -----------  -----------  ---------------------------  ------------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  ---------------------------------------------------------------------------------------
    multinomial  multinomial  Ridge ( lambda = 0.007732 )  nlambda = 30, lambda.max = 21.707, lambda.min = 0.007732, lambda.1se = 0.01062  8430                          8424                           205                     AutoML_1_20230909_00442_training_Key_Frame__upload_9b0ad4f7d1eb0254f20700a6c8f745c0.hex

ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.1832485926285127
RMSE: 0.4280754520274582
LogLoss: 0.5848479289901717
Null degrees of freedom: 20190
Residual degrees of freedom: 11761
Null deviance: 63923.76920676409
Residual deviance: 23617.327619764204
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
anger    contempt    disgust    happy    neutral    surprise    Error      Rate
-------  ----------  ---------  -------  ---------  ----------  ---------  --------------
2146     420         0          18       46         529         0.320671   1,013 / 3,159
429      1948        0          47       84         349         0.318166   909 / 2,857
1        0           0          0        0          0           1          1 / 1
12       23          0          4407     553        43          0.125248   631 / 5,038
46       38          0          316      4614       103         0.0982998  503 / 5,117
383      360         0          104      325        2847        0.291615   1,172 / 4,019
3017     2789        0          4892     5622       3871        0.20945    4,229 / 20,191

Top-6 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.79055
2    0.942301
3    0.984349
4    0.997524
5    0.999851
6    1

ModelMetricsMultinomialGLM: glm
** Reported on cross-validation data. **

MSE: 0.18351216993959682
RMSE: 0.42838320454891415
LogLoss: 0.588139110499819
Null degrees of freedom: 20190
Residual degrees of freedom: 11761
Null deviance: Infinity
Residual deviance: 23750.499560828222
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
anger    contempt    disgust    happy    neutral    surprise    Error      Rate
-------  ----------  ---------  -------  ---------  ----------  ---------  --------------
2125     436         0          17       46         535         0.327319   1,034 / 3,159
407      1972        0          46       84         348         0.309765   885 / 2,857
1        0           0          0        0          0           1          1 / 1
11       22          0          4405     554        46          0.125645   633 / 5,038
46       38          0          316      4610       107         0.0990815  507 / 5,117
389      361         0          100      323        2846        0.291864   1,173 / 4,019
2979     2829 

In [24]:
aml.distribution

'AUTO'

In [8]:
lb = aml.leaderboard
lb.head(rows=15)

model_id,mean_per_class_error,logloss,rmse,mse
GLM_1_AutoML_1_20230909_00442,0.358946,0.588139,0.428383,0.183512
XGBoost_1_AutoML_1_20230909_00442,0.384425,0.629443,0.435483,0.189645


In [13]:
test_h2o_frame = test_h2o_frame.drop('col1404')
pred = aml.predict(test_h2o_frame)

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [14]:
pred


predict,anger,contempt,disgust,happy,neutral,surprise
anger,0.785474,0.00315324,7.16687e-07,0.000155655,2.75566e-06,0.211214
anger,0.513401,0.224812,1.63442e-05,7.30381e-05,0.00407176,0.257626
anger,0.915206,0.0457143,4.95148e-06,1.73313e-05,0.0040324,0.0350254
anger,0.858818,0.00353934,1.40306e-06,3.76374e-05,1.14827e-05,0.137592
contempt,0.260427,0.48689,2.40392e-05,0.00255855,0.0118256,0.238274
anger,0.732838,0.091804,1.497e-05,7.53426e-05,0.0174777,0.15779
anger,0.828043,0.0637554,3.51999e-07,3.13452e-07,0.000490878,0.10771
anger,0.889751,0.106978,2.23184e-06,5.22006e-05,4.08157e-05,0.00317531
anger,0.56497,0.178844,7.34457e-06,0.000578934,0.038937,0.216663
anger,0.326414,0.152872,5.89797e-05,0.00278635,0.19269,0.325179


In [21]:
best_model = aml.leader
# aml.score(test_h2o_frame, test_h2o_frame.MEDV)


In [23]:
h2o.save_model(model=best_model, path="./")

'/home/vorkov/Workspace/Emotion-Decetion-Service/learning/GLM_1_AutoML_1_20230909_00442'

In [24]:
h2o.load_model('/home/vorkov/Workspace/Emotion-Decetion-Service/learning/GLM_1_AutoML_1_20230909_00442')

Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_1_AutoML_1_20230909_00442


GLM Model: summary
    family       link         regularization               lambda_search                                                                   number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  -----------  -----------  ---------------------------  ------------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  ---------------------------------------------------------------------------------------
    multinomial  multinomial  Ridge ( lambda = 0.007732 )  nlambda = 30, lambda.max = 21.707, lambda.min = 0.007732, lambda.1se = 0.01062  8430                          8424                           205                     AutoML_1_20230909_00442_training_Key_Frame__upload_9b0ad4f7d1eb0254f20700a6c8f745c0.hex

ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.1832485926285127
RMSE: 0.4280754520274582
LogLoss: 0.5848479289901717
Null degrees of freedom: 20190
Residual degrees of freedom: 11761
Null deviance: 63923.76920676409
Residual deviance: 23617.327619764204
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
anger    contempt    disgust    happy    neutral    surprise    Error      Rate
-------  ----------  ---------  -------  ---------  ----------  ---------  --------------
2146     420         0          18       46         529         0.320671   1,013 / 3,159
429      1948        0          47       84         349         0.318166   909 / 2,857
1        0           0          0        0          0           1          1 / 1
12       23          0          4407     553        43          0.125248   631 / 5,038
46       38          0          316      4614       103         0.0982998  503 / 5,117
383      360         0          104      325        2847        0.291615   1,172 / 4,019
3017     2789        0          4892     5622       3871        0.20945    4,229 / 20,191

Top-6 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.79055
2    0.942301
3    0.984349
4    0.997524
5    0.999851
6    1

ModelMetricsMultinomialGLM: glm
** Reported on cross-validation data. **

MSE: 0.18351216993959682
RMSE: 0.42838320454891415
LogLoss: 0.588139110499819
Null degrees of freedom: 20190
Residual degrees of freedom: 11761
Null deviance: Infinity
Residual deviance: 23750.499560828222
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
anger    contempt    disgust    happy    neutral    surprise    Error      Rate
-------  ----------  ---------  -------  ---------  ----------  ---------  --------------
2125     436         0          17       46         535         0.327319   1,034 / 3,159
407      1972        0          46       84         348         0.309765   885 / 2,857
1        0           0          0        0          0           1          1 / 1
11       22          0          4405     554        46          0.125645   633 / 5,038
46       38          0          316      4610       107         0.0990815  507 / 5,117
389      361         0          100      323        2846        0.291864   1,173 / 4,019
2979     2829 